In [237]:
import os
import sys
sys.path.insert(0, os.path.abspath(os.path.join('../..', 'src')))
sys.setrecursionlimit(20500)
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import make_dataset as mk

import tools as tools

import pickle
import numpy as np

In [238]:
'''import data'''

main_strategy_1 = pd.read_csv(os.path.abspath(os.path.join('../..', 'data/processed/encoded_labels'))+'/main_strategy_1.csv')
first_8= pd.read_csv(os.path.abspath(os.path.join('../..', 'data/processed/encoded_labels'))+'/first_8.csv')
second_8= pd.read_csv(os.path.abspath(os.path.join('../..', 'data/processed/encoded_labels'))+'/second_8.csv')
third_8= pd.read_csv(os.path.abspath(os.path.join('../..', 'data/processed/encoded_labels'))+'/third_8.csv')
rest_7= pd.read_csv(os.path.abspath(os.path.join('../..', 'data/processed/encoded_labels'))+'/rest_7.csv')

main_strategy_2= pd.read_csv(os.path.abspath(os.path.join('../..', 'data/processed/encoded_labels'))+'/main_strategy_2.csv')
first_9= pd.read_csv(os.path.abspath(os.path.join('../..', 'data/processed/encoded_labels'))+'/first_9.csv')
second_9= pd.read_csv(os.path.abspath(os.path.join('../..', 'data/processed/encoded_labels'))+'/second_9.csv')
third_9= pd.read_csv(os.path.abspath(os.path.join('../..', 'data/processed/encoded_labels'))+'/third_9.csv')
rest_8= pd.read_csv(os.path.abspath(os.path.join('../..', 'data/processed/encoded_labels'))+'/rest_8.csv')

main_strategy_3= pd.read_csv(os.path.abspath(os.path.join('../..', 'data/processed/encoded_labels'))+'/main_strategy_3.csv')
first_10= pd.read_csv(os.path.abspath(os.path.join('../..', 'data/processed/encoded_labels'))+'/first_10.csv')
second_10= pd.read_csv(os.path.abspath(os.path.join('../..', 'data/processed/encoded_labels'))+'/second_10.csv')
third_10= pd.read_csv(os.path.abspath(os.path.join('../..', 'data/processed/encoded_labels'))+'/third_10.csv')
rest_9= pd.read_csv(os.path.abspath(os.path.join('../..', 'data/processed/encoded_labels'))+'/rest_9.csv')

In [239]:
'''count and re-process labels'''
main_strategy_1, df_lambda_ms_1 = tools.count_labels(main_strategy_1)
main_strategy_2, df_lambda_ms_2 = tools.count_labels(main_strategy_2)
main_strategy_3, df_lambda_ms_3 = tools.count_labels(main_strategy_3)

Shape:(10, 2)


,category,Count
0,capacity_building,113
1,enabling,45
2,finance_economy,32
3,food_and_agricultural_commodities,17
4,governance,131
5,law_regulation,120
6,management_operation,104
7,mitigation_adaptation,3
8,monitor_inventory,63
9,technology_innovation,20


Shape:(11, 2)


,category,Count
0,Unnamed: 4,4
1,capacity_building,142
2,enabling,20
3,finance_economy,56
4,food_and_agricultural_commodities,27
5,governance,93
6,law_regulation,115
7,management_operation,89
8,mitigation_adaptation,22
9,monitor_inventory,54


Shape:(11, 2)


,category,Count
0,Unnamed: 4,78
1,capacity_building,163
2,enabling,22
3,finance_economy,81
4,food_and_agricultural_commodities,46
5,governance,23
6,law_regulation,27
7,management_operation,91
8,mitigation_adaptation,21
9,monitor_inventory,61


In [240]:
# delete rows with non-existing categories:
strategies = ['capacity_building', 'enabling',  'finance_economy',
       'food_and_agricultural_commodities', 'governance', 'law_regulation',
       'management_operation', 'mitigation_adaptation', 'monitor_inventory','technology_innovation']

main_strategy_1 = main_strategy_1[main_strategy_1['category'].isin(strategies)]
main_strategy_2 = main_strategy_2[main_strategy_2['category'].isin(strategies)]
main_strategy_3 = main_strategy_3[main_strategy_3['category'].isin(strategies)]

In [241]:
relevant = main_strategy_1[['PIMS_ID', 'capacity_building', 'enabling',  'finance_economy',
       'food_and_agricultural_commodities', 'governance', 'law_regulation',
       'management_operation', 'mitigation_adaptation', 'monitor_inventory','technology_innovation', 'category']]

relevant.rename({'category':'category_1'}, axis = 1, inplace = True)
              
result = pd.concat([relevant, main_strategy_2['category']], axis=1, sort=False)
result.rename({'category':'category_2'}, axis = 1, inplace = True)

result = pd.concat([result, main_strategy_3['category']], axis=1, sort=False)
result.rename({'category':'category_3'}, axis = 1, inplace = True)

# turn to multi_label problem

result[['capacity_building', 'enabling',  'finance_economy',
       'food_and_agricultural_commodities', 'governance', 'law_regulation',
       'management_operation', 'mitigation_adaptation', 'monitor_inventory','technology_innovation']] = result[['capacity_building', 'enabling',  'finance_economy',
       'food_and_agricultural_commodities', 'governance', 'law_regulation',
       'management_operation', 'mitigation_adaptation', 'monitor_inventory','technology_innovation']].fillna(0).astype(int)

def to_multi_label(dataframe):
    for index, row in dataframe.iterrows():

        for column in dataframe.columns:

            if row['category_2'] == column:

                if row[column] == 0:
                    dataframe.at[index, column] = 1
            else:
                pass

            if row['category_3'] == column:

                if row[column] == 0:
                    dataframe.at[index, column] = 1
            else:
                pass 
    
    
    dataframe['labels']= dataframe[['capacity_building', 'enabling',  'finance_economy',
       'food_and_agricultural_commodities', 'governance', 'law_regulation',
       'management_operation', 'mitigation_adaptation', 'monitor_inventory','technology_innovation']].fillna(0).astype(int).values.tolist()
    
    return dataframe

to_multi_label(result)
result = result.reset_index(drop=True)

In [279]:
'''first triplet'''

land_num_1 = first_8._get_numeric_data()
land_num_df_1 = land_num_1.drop(columns=['all_text_clean', 'all_text_clean_spacy'])

land_num_2 = second_8._get_numeric_data()
land_num_df_2 = land_num_2.drop(columns=['all_text_clean', 'all_text_clean_spacy', 'Unnamed: 4'])

land_num_3 = third_8._get_numeric_data()
land_num_df_3 = land_num_3.drop(columns=['all_text_clean', 'all_text_clean_spacy', 'Unnamed: 4'])

'''combine lists'''
land_num_1 = land_num_df_1.drop(columns=['PIMS_ID'])
land_num_2 = land_num_df_2.drop(columns=['PIMS_ID'])
land_num_3 = land_num_df_3.drop(columns=['PIMS_ID'])

land_sub_1 = land_num_1.columns.tolist()
land_sub_2 = land_num_2.columns.tolist()
land_sub_3 = land_num_3.columns.tolist()

sub = land_sub_1 + list(set(land_sub_2) - set(land_sub_1))
sub = sub + list(set(land_sub_3) - set(sub))

unmatched_items_1_2 = [d for d in land_sub_1 if d not in land_sub_2]
unmatched_items_1_3 = [d for d in land_sub_1 if d not in land_sub_3]

unmatched_items_2_1 = [d for d in land_sub_2 if d not in land_sub_1]
unmatched_items_2_3 = [d for d in land_sub_2 if d not in land_sub_3]

unmatched_items_3_1 = [d for d in land_sub_3 if d not in land_sub_1]
unmatched_items_3_2 = [d for d in land_sub_3 if d not in land_sub_2]

for col in unmatched_items_1_2:
    land_num_df_2[col] = 0
for col in unmatched_items_1_3:
    land_num_df_3[col] = 0
    
for col in unmatched_items_2_1:
    land_num_df_1[col] = 0
for col in unmatched_items_2_3:
    land_num_df_3[col] = 0
    
    
for col in unmatched_items_3_1:
    land_num_df_1[col] = 0
for col in unmatched_items_3_2:
    land_num_df_2[col] = 0
    
sub_category = land_num_df_1.merge(land_num_df_2, how='left', left_on=['PIMS_ID'], right_on=['PIMS_ID'])
sub_category = sub_category.merge(land_num_df_3, how='left', left_on=['PIMS_ID'], right_on=['PIMS_ID'])

for index, row in sub_category.iterrows():
    for i in sub:          
        if row[i+"_x"] or row[i+"_y"]:
            sub_category.at[index, i] = 1
        else:
            sub_category.at[index, i] = 0     
            
for i in sub:
    sub_category= sub_category.drop(columns=[i+"_x"])
    sub_category= sub_category.drop(columns=[i+"_y"])

sub_category_1 = sub_category    
#sub_category = sub_category.drop(columns=['PIMS_ID'])

In [280]:
'''second triplet'''

land_num_1 = first_9._get_numeric_data()
land_num_df_1 = land_num_1.drop(columns=['all_text_clean', 'all_text_clean_spacy', 'Unnamed: 4'])

land_num_2 = second_9._get_numeric_data()
land_num_df_2 = land_num_2.drop(columns=['all_text_clean', 'all_text_clean_spacy', 'Unnamed: 4'])

land_num_3 = third_9._get_numeric_data()
land_num_df_3 = land_num_3.drop(columns=['all_text_clean', 'all_text_clean_spacy','Unnamed: 4'])

'''combine lists'''
land_num_1 = land_num_df_1.drop(columns=['PIMS_ID'])
land_num_2 = land_num_df_2.drop(columns=['PIMS_ID'])
land_num_3 = land_num_df_3.drop(columns=['PIMS_ID'])

land_sub_1 = land_num_1.columns.tolist()
land_sub_2 = land_num_2.columns.tolist()
land_sub_3 = land_num_3.columns.tolist()

sub = land_sub_1 + list(set(land_sub_2) - set(land_sub_1))
sub = sub + list(set(land_sub_3) - set(sub))

unmatched_items_1_2 = [d for d in land_sub_1 if d not in land_sub_2]
unmatched_items_1_3 = [d for d in land_sub_1 if d not in land_sub_3]

unmatched_items_2_1 = [d for d in land_sub_2 if d not in land_sub_1]
unmatched_items_2_3 = [d for d in land_sub_2 if d not in land_sub_3]

unmatched_items_3_1 = [d for d in land_sub_3 if d not in land_sub_1]
unmatched_items_3_2 = [d for d in land_sub_3 if d not in land_sub_2]

for col in unmatched_items_1_2:
    land_num_df_2[col] = 0
for col in unmatched_items_1_3:
    land_num_df_3[col] = 0
    
for col in unmatched_items_2_1:
    land_num_df_1[col] = 0
for col in unmatched_items_2_3:
    land_num_df_3[col] = 0
    
    
for col in unmatched_items_3_1:
    land_num_df_1[col] = 0
for col in unmatched_items_3_2:
    land_num_df_2[col] = 0
    
sub_category = land_num_df_1.merge(land_num_df_2, how='left', left_on=['PIMS_ID'], right_on=['PIMS_ID'])
sub_category = sub_category.merge(land_num_df_3, how='left', left_on=['PIMS_ID'], right_on=['PIMS_ID'])

for index, row in sub_category.iterrows():
    for i in sub:          
        if row[i+"_x"] or row[i+"_y"]:
            sub_category.at[index, i] = 1
        else:
            sub_category.at[index, i] = 0     
            
for i in sub:
    sub_category= sub_category.drop(columns=[i+"_x"])
    sub_category= sub_category.drop(columns=[i+"_y"])

sub_category_2 = sub_category    
#sub_category_2 = sub_category.drop(columns=['PIMS_ID'])

In [281]:
'''third triplet'''

land_num_1 = first_10._get_numeric_data()
land_num_df_1 = land_num_1.drop(columns=['all_text_clean', 'all_text_clean_spacy', 'Unnamed: 4'])

land_num_2 = second_10._get_numeric_data()
land_num_df_2 = land_num_2.drop(columns=['all_text_clean', 'all_text_clean_spacy', 'Unnamed: 4'])

land_num_3 = third_10._get_numeric_data()
land_num_df_3 = land_num_3.drop(columns=['all_text_clean', 'all_text_clean_spacy', 'Unnamed: 4'])

'''combine lists'''
land_num_1 = land_num_df_1.drop(columns=['PIMS_ID'])
land_num_2 = land_num_df_2.drop(columns=['PIMS_ID'])
land_num_3 = land_num_df_3.drop(columns=['PIMS_ID'])

land_sub_1 = land_num_1.columns.tolist()
land_sub_2 = land_num_2.columns.tolist()
land_sub_3 = land_num_3.columns.tolist()

sub = land_sub_1 + list(set(land_sub_2) - set(land_sub_1))
sub = sub + list(set(land_sub_3) - set(sub))

unmatched_items_1_2 = [d for d in land_sub_1 if d not in land_sub_2]
unmatched_items_1_3 = [d for d in land_sub_1 if d not in land_sub_3]

unmatched_items_2_1 = [d for d in land_sub_2 if d not in land_sub_1]
unmatched_items_2_3 = [d for d in land_sub_2 if d not in land_sub_3]

unmatched_items_3_1 = [d for d in land_sub_3 if d not in land_sub_1]
unmatched_items_3_2 = [d for d in land_sub_3 if d not in land_sub_2]

for col in unmatched_items_1_2:
    land_num_df_2[col] = 0
for col in unmatched_items_1_3:
    land_num_df_3[col] = 0
    
for col in unmatched_items_2_1:
    land_num_df_1[col] = 0
for col in unmatched_items_2_3:
    land_num_df_3[col] = 0
    
    
for col in unmatched_items_3_1:
    land_num_df_1[col] = 0
for col in unmatched_items_3_2:
    land_num_df_2[col] = 0
    
sub_category = land_num_df_1.merge(land_num_df_2, how='left', left_on=['PIMS_ID'], right_on=['PIMS_ID'])
sub_category = sub_category.merge(land_num_df_3, how='left', left_on=['PIMS_ID'], right_on=['PIMS_ID'])

for index, row in sub_category.iterrows():
    for i in sub:          
        if row[i+"_x"] or row[i+"_y"]:
            sub_category.at[index, i] = 1
        else:
            sub_category.at[index, i] = 0     
            
for i in sub:
    sub_category= sub_category.drop(columns=[i+"_x"])
    sub_category= sub_category.drop(columns=[i+"_y"])

sub_category_3 = sub_category       
#sub_category_3 = sub_category.drop(columns=['PIMS_ID'])

In [282]:
'''fourth triplet'''

land_num_1 = rest_7._get_numeric_data()
land_num_df_1 = land_num_1.drop(columns=['all_text_clean', 'all_text_clean_spacy', 'Unnamed: 4'])

land_num_2 = rest_8._get_numeric_data()
land_num_df_2 = land_num_2.drop(columns=['all_text_clean', 'all_text_clean_spacy', 'Unnamed: 4'])

land_num_3 = rest_9._get_numeric_data()
land_num_df_3 = land_num_3.drop(columns=['all_text_clean', 'all_text_clean_spacy', 'Unnamed: 4'])

'''combine lists'''
land_num_1 = land_num_df_1.drop(columns=['PIMS_ID'])
land_num_2 = land_num_df_2.drop(columns=['PIMS_ID'])
land_num_3 = land_num_df_3.drop(columns=['PIMS_ID'])

land_sub_1 = land_num_1.columns.tolist()
land_sub_2 = land_num_2.columns.tolist()
land_sub_3 = land_num_3.columns.tolist()

sub = land_sub_1 + list(set(land_sub_2) - set(land_sub_1))
sub = sub + list(set(land_sub_3) - set(sub))

unmatched_items_1_2 = [d for d in land_sub_1 if d not in land_sub_2]
unmatched_items_1_3 = [d for d in land_sub_1 if d not in land_sub_3]

unmatched_items_2_1 = [d for d in land_sub_2 if d not in land_sub_1]
unmatched_items_2_3 = [d for d in land_sub_2 if d not in land_sub_3]

unmatched_items_3_1 = [d for d in land_sub_3 if d not in land_sub_1]
unmatched_items_3_2 = [d for d in land_sub_3 if d not in land_sub_2]

for col in unmatched_items_1_2:
    land_num_df_2[col] = 0
for col in unmatched_items_1_3:
    land_num_df_3[col] = 0
    
for col in unmatched_items_2_1:
    land_num_df_1[col] = 0
for col in unmatched_items_2_3:
    land_num_df_3[col] = 0
    
    
for col in unmatched_items_3_1:
    land_num_df_1[col] = 0
for col in unmatched_items_3_2:
    land_num_df_2[col] = 0
    
sub_category = land_num_df_1.merge(land_num_df_2, how='left', left_on=['PIMS_ID'], right_on=['PIMS_ID'])
sub_category = sub_category.merge(land_num_df_3, how='left', left_on=['PIMS_ID'], right_on=['PIMS_ID'])

for index, row in sub_category.iterrows():
    for i in sub:          
        if row[i+"_x"] or row[i+"_y"]:
            sub_category.at[index, i] = 1
        else:
            sub_category.at[index, i] = 0     
            
for i in sub:
    sub_category= sub_category.drop(columns=[i+"_x"])
    sub_category= sub_category.drop(columns=[i+"_y"])
    
sub_category_4 = sub_category    
#sub_category_4 = sub_category.drop(columns=['PIMS_ID'])

In [283]:
'''merge first three triplets'''
'''fourth triplet'''

land_num_1 = sub_category_1._get_numeric_data()
land_num_df_1 = land_num_1

land_num_2 = sub_category_2._get_numeric_data()
land_num_df_2 = land_num_2

land_num_3 = sub_category_3._get_numeric_data()
land_num_df_3 = land_num_3

'''combine lists'''
land_num_1 = land_num_df_1.drop(columns=['PIMS_ID'])
land_num_2 = land_num_df_2.drop(columns=['PIMS_ID'])
land_num_3 = land_num_df_3.drop(columns=['PIMS_ID'])

land_sub_1 = land_num_1.columns.tolist()
land_sub_2 = land_num_2.columns.tolist()
land_sub_3 = land_num_3.columns.tolist()

sub = land_sub_1 + list(set(land_sub_2) - set(land_sub_1))
sub = sub + list(set(land_sub_3) - set(sub))

unmatched_items_1_2 = [d for d in land_sub_1 if d not in land_sub_2]
unmatched_items_1_3 = [d for d in land_sub_1 if d not in land_sub_3]

unmatched_items_2_1 = [d for d in land_sub_2 if d not in land_sub_1]
unmatched_items_2_3 = [d for d in land_sub_2 if d not in land_sub_3]

unmatched_items_3_1 = [d for d in land_sub_3 if d not in land_sub_1]
unmatched_items_3_2 = [d for d in land_sub_3 if d not in land_sub_2]

for col in unmatched_items_1_2:
    land_num_df_2[col] = 0
for col in unmatched_items_1_3:
    land_num_df_3[col] = 0
    
for col in unmatched_items_2_1:
    land_num_df_1[col] = 0
for col in unmatched_items_2_3:
    land_num_df_3[col] = 0
    
    
for col in unmatched_items_3_1:
    land_num_df_1[col] = 0
for col in unmatched_items_3_2:
    land_num_df_2[col] = 0
    
sub_category = land_num_df_1.merge(land_num_df_2, how='left', left_on=['PIMS_ID'], right_on=['PIMS_ID'])
sub_category = sub_category.merge(land_num_df_3, how='left', left_on=['PIMS_ID'], right_on=['PIMS_ID'])

for index, row in sub_category.iterrows():
    for i in sub:          
        if row[i+"_x"] or row[i+"_y"]:
            sub_category.at[index, i] = 1
        else:
            sub_category.at[index, i] = 0     
            
for i in sub:
    sub_category= sub_category.drop(columns=[i+"_x"])
    sub_category= sub_category.drop(columns=[i+"_y"])
    
sub_category_all_1 = sub_category

In [289]:
df_num_1 = sub_category_4._get_numeric_data()
df_num_1_1 = df_num_1
df_num_2 = sub_category_all_1._get_numeric_data()
df_num_2_1 = df_num_2


'''combine lists'''
df_num_1 = df_num_1.drop(columns=['PIMS_ID'])
df_num_2 = df_num_2.drop(columns=['PIMS_ID'])
sub_sub_1 = df_num_1.columns.tolist()
sub_sub_2 = df_num_2.columns.tolist()
sub_sub = sub_sub_1 + list(set(sub_sub_2) - set(sub_sub_1))

unmatched_items_1 = [d for d in sub_sub_1 if d not in sub_sub_2]
unmatched_items_2 = [d for d in sub_sub_2 if d not in sub_sub_1]

for col in unmatched_items_1:
    df_num_2_1[col] = 0
    
for col in unmatched_items_2:
    df_num_1_1[col] = 0
    
sub_category = df_num_1_1.merge(df_num_2_1, how='left', left_on=['PIMS_ID'], right_on=['PIMS_ID'])

for index, row in sub_category.iterrows():
    for i in sub_sub:          
        if row[i+"_x"] or row[i+"_y"]:
            sub_category.at[index, i] = 1
        else:
            sub_category.at[index, i] = 0     
            
for i in sub_sub:
    sub_category= sub_category.drop(columns=[i+"_x"])
    sub_category= sub_category.drop(columns=[i+"_y"])
    
sub_strategies = sub_category.drop(columns=['PIMS_ID'])

In [290]:
'''merge with main strategies'''
strategies = result.drop(columns=['category_1', 'category_2', 'category_3', 'labels'])
strategies = pd.concat([strategies, sub_strategies], axis=1)

In [291]:
'''merge with one of the cleaned dataframes'''
df = pd.read_csv(os.path.abspath(os.path.join('../..', 'data/processed/'))+"/technical_team.csv")

In [292]:
df = df[['PIMS_ID', 'all_text_clean', 'all_text_clean_spacy']]

In [293]:
strategy = df.merge(strategies, how='left', left_on=['PIMS_ID'], right_on=['PIMS_ID'])
strategy.to_csv(os.path.abspath(os.path.join('../..', 'data/processed/'))+"/strategies.csv")

## Dictionaries:

## First Strategy

In [294]:
'''first triplet'''

land_num_1 = first_8._get_numeric_data()
land_num_df_1 = land_num_1.drop(columns=['all_text_clean', 'all_text_clean_spacy'])

land_num_2 = second_8._get_numeric_data()
land_num_df_2 = land_num_2.drop(columns=['all_text_clean', 'all_text_clean_spacy', 'Unnamed: 4'])

land_num_3 = third_8._get_numeric_data()
land_num_df_3 = land_num_3.drop(columns=['all_text_clean', 'all_text_clean_spacy', 'Unnamed: 4'])

'''combine lists'''
land_num_1 = land_num_df_1.drop(columns=['PIMS_ID'])
land_num_2 = land_num_df_2.drop(columns=['PIMS_ID'])
land_num_3 = land_num_df_3.drop(columns=['PIMS_ID'])

land_sub_1 = land_num_1.columns.tolist()
land_sub_2 = land_num_2.columns.tolist()
land_sub_3 = land_num_3.columns.tolist()

sub = land_sub_1 + list(set(land_sub_2) - set(land_sub_1))
sub = sub + list(set(land_sub_3) - set(sub))

unmatched_items_1_2 = [d for d in land_sub_1 if d not in land_sub_2]
unmatched_items_1_3 = [d for d in land_sub_1 if d not in land_sub_3]

unmatched_items_2_1 = [d for d in land_sub_2 if d not in land_sub_1]
unmatched_items_2_3 = [d for d in land_sub_2 if d not in land_sub_3]

unmatched_items_3_1 = [d for d in land_sub_3 if d not in land_sub_1]
unmatched_items_3_2 = [d for d in land_sub_3 if d not in land_sub_2]

for col in unmatched_items_1_2:
    land_num_df_2[col] = 0
for col in unmatched_items_1_3:
    land_num_df_3[col] = 0
    
for col in unmatched_items_2_1:
    land_num_df_1[col] = 0
for col in unmatched_items_2_3:
    land_num_df_3[col] = 0
    
    
for col in unmatched_items_3_1:
    land_num_df_1[col] = 0
for col in unmatched_items_3_2:
    land_num_df_2[col] = 0
    
sub_category = land_num_df_1.merge(land_num_df_2, how='left', left_on=['PIMS_ID'], right_on=['PIMS_ID'])
sub_category = sub_category.merge(land_num_df_3, how='left', left_on=['PIMS_ID'], right_on=['PIMS_ID'])

for index, row in sub_category.iterrows():
    for i in sub:          
        if row[i+"_x"] or row[i+"_y"]:
            sub_category.at[index, i] = 1
        else:
            sub_category.at[index, i] = 0     
            
for i in sub:
    sub_category= sub_category.drop(columns=[i+"_x"])
    sub_category= sub_category.drop(columns=[i+"_y"])
    
df_num_1 = sub_category._get_numeric_data()
df_num_1_1 = df_num_1
df_num_2 = rest_7._get_numeric_data()
df_num_2_1 = df_num_2.drop(columns=['all_text_clean', 'all_text_clean_spacy', 'Unnamed: 4'])


'''combine lists'''
df_num_1 = df_num_1_1.drop(columns=['PIMS_ID'])
df_num_2 = df_num_2_1.drop(columns=['PIMS_ID'])
sub_sub_1 = df_num_1.columns.tolist()
sub_sub_2 = df_num_2.columns.tolist()
sub_sub = sub_sub_1 + list(set(sub_sub_2) - set(sub_sub_1))

unmatched_items_1 = [d for d in sub_sub_1 if d not in sub_sub_2]
unmatched_items_2 = [d for d in sub_sub_2 if d not in sub_sub_1]

for col in unmatched_items_1:
    df_num_2_1[col] = 0
    
for col in unmatched_items_2:
    df_num_1_1[col] = 0
    
sub_category = df_num_1_1.merge(df_num_2_1, how='left', left_on=['PIMS_ID'], right_on=['PIMS_ID'])

for index, row in sub_category.iterrows():
    for i in sub_sub:          
        if row[i+"_x"] or row[i+"_y"]:
            sub_category.at[index, i] = 1
        else:
            sub_category.at[index, i] = 0     
            
for i in sub_sub:
    sub_category= sub_category.drop(columns=[i+"_x"])
    sub_category= sub_category.drop(columns=[i+"_y"])
    
sub_strategis_1 = sub_category

In [295]:
'''second triplet'''

land_num_1 = first_9._get_numeric_data()
land_num_df_1 = land_num_1.drop(columns=['all_text_clean', 'all_text_clean_spacy', 'Unnamed: 4'])

land_num_2 = second_9._get_numeric_data()
land_num_df_2 = land_num_2.drop(columns=['all_text_clean', 'all_text_clean_spacy', 'Unnamed: 4'])

land_num_3 = third_9._get_numeric_data()
land_num_df_3 = land_num_3.drop(columns=['all_text_clean', 'all_text_clean_spacy', 'Unnamed: 4'])

'''combine lists'''
land_num_1 = land_num_df_1.drop(columns=['PIMS_ID'])
land_num_2 = land_num_df_2.drop(columns=['PIMS_ID'])
land_num_3 = land_num_df_3.drop(columns=['PIMS_ID'])

land_sub_1 = land_num_1.columns.tolist()
land_sub_2 = land_num_2.columns.tolist()
land_sub_3 = land_num_3.columns.tolist()

sub = land_sub_1 + list(set(land_sub_2) - set(land_sub_1))
sub = sub + list(set(land_sub_3) - set(sub))

unmatched_items_1_2 = [d for d in land_sub_1 if d not in land_sub_2]
unmatched_items_1_3 = [d for d in land_sub_1 if d not in land_sub_3]

unmatched_items_2_1 = [d for d in land_sub_2 if d not in land_sub_1]
unmatched_items_2_3 = [d for d in land_sub_2 if d not in land_sub_3]

unmatched_items_3_1 = [d for d in land_sub_3 if d not in land_sub_1]
unmatched_items_3_2 = [d for d in land_sub_3 if d not in land_sub_2]

for col in unmatched_items_1_2:
    land_num_df_2[col] = 0
for col in unmatched_items_1_3:
    land_num_df_3[col] = 0
    
for col in unmatched_items_2_1:
    land_num_df_1[col] = 0
for col in unmatched_items_2_3:
    land_num_df_3[col] = 0
    
    
for col in unmatched_items_3_1:
    land_num_df_1[col] = 0
for col in unmatched_items_3_2:
    land_num_df_2[col] = 0
    
sub_category = land_num_df_1.merge(land_num_df_2, how='left', left_on=['PIMS_ID'], right_on=['PIMS_ID'])
sub_category = sub_category.merge(land_num_df_3, how='left', left_on=['PIMS_ID'], right_on=['PIMS_ID'])

for index, row in sub_category.iterrows():
    for i in sub:          
        if row[i+"_x"] or row[i+"_y"]:
            sub_category.at[index, i] = 1
        else:
            sub_category.at[index, i] = 0     
            
for i in sub:
    sub_category= sub_category.drop(columns=[i+"_x"])
    sub_category= sub_category.drop(columns=[i+"_y"])
    
df_num_1 = sub_category._get_numeric_data()
df_num_1_1 = df_num_1
df_num_2 = rest_8._get_numeric_data()
df_num_2_1 = df_num_2.drop(columns=['all_text_clean', 'all_text_clean_spacy', 'Unnamed: 4'])


'''combine lists'''
df_num_1 = df_num_1_1.drop(columns=['PIMS_ID'])
df_num_2 = df_num_2_1.drop(columns=['PIMS_ID'])
sub_sub_1 = df_num_1.columns.tolist()
sub_sub_2 = df_num_2.columns.tolist()
sub_sub = sub_sub_1 + list(set(sub_sub_2) - set(sub_sub_1))

unmatched_items_1 = [d for d in sub_sub_1 if d not in sub_sub_2]
unmatched_items_2 = [d for d in sub_sub_2 if d not in sub_sub_1]

for col in unmatched_items_1:
    df_num_2_1[col] = 0
    
for col in unmatched_items_2:
    df_num_1_1[col] = 0
    
sub_category = df_num_1_1.merge(df_num_2_1, how='left', left_on=['PIMS_ID'], right_on=['PIMS_ID'])

for index, row in sub_category.iterrows():
    for i in sub_sub:          
        if row[i+"_x"] or row[i+"_y"]:
            sub_category.at[index, i] = 1
        else:
            sub_category.at[index, i] = 0     
            
for i in sub_sub:
    sub_category= sub_category.drop(columns=[i+"_x"])
    sub_category= sub_category.drop(columns=[i+"_y"])
    
sub_strategis_2 = sub_category

In [296]:
'''second triplet'''

land_num_1 = first_10._get_numeric_data()
land_num_df_1 = land_num_1.drop(columns=['all_text_clean', 'all_text_clean_spacy', 'Unnamed: 4'])

land_num_2 = second_10._get_numeric_data()
land_num_df_2 = land_num_2.drop(columns=['all_text_clean', 'all_text_clean_spacy', 'Unnamed: 4'])

land_num_3 = third_10._get_numeric_data()
land_num_df_3 = land_num_3.drop(columns=['all_text_clean', 'all_text_clean_spacy', 'Unnamed: 4'])

'''combine lists'''
land_num_1 = land_num_df_1.drop(columns=['PIMS_ID'])
land_num_2 = land_num_df_2.drop(columns=['PIMS_ID'])
land_num_3 = land_num_df_3.drop(columns=['PIMS_ID'])

land_sub_1 = land_num_1.columns.tolist()
land_sub_2 = land_num_2.columns.tolist()
land_sub_3 = land_num_3.columns.tolist()

sub = land_sub_1 + list(set(land_sub_2) - set(land_sub_1))
sub = sub + list(set(land_sub_3) - set(sub))

unmatched_items_1_2 = [d for d in land_sub_1 if d not in land_sub_2]
unmatched_items_1_3 = [d for d in land_sub_1 if d not in land_sub_3]

unmatched_items_2_1 = [d for d in land_sub_2 if d not in land_sub_1]
unmatched_items_2_3 = [d for d in land_sub_2 if d not in land_sub_3]

unmatched_items_3_1 = [d for d in land_sub_3 if d not in land_sub_1]
unmatched_items_3_2 = [d for d in land_sub_3 if d not in land_sub_2]

for col in unmatched_items_1_2:
    land_num_df_2[col] = 0
for col in unmatched_items_1_3:
    land_num_df_3[col] = 0
    
for col in unmatched_items_2_1:
    land_num_df_1[col] = 0
for col in unmatched_items_2_3:
    land_num_df_3[col] = 0
    
    
for col in unmatched_items_3_1:
    land_num_df_1[col] = 0
for col in unmatched_items_3_2:
    land_num_df_2[col] = 0
    
sub_category = land_num_df_1.merge(land_num_df_2, how='left', left_on=['PIMS_ID'], right_on=['PIMS_ID'])
sub_category = sub_category.merge(land_num_df_3, how='left', left_on=['PIMS_ID'], right_on=['PIMS_ID'])

for index, row in sub_category.iterrows():
    for i in sub:          
        if row[i+"_x"] or row[i+"_y"]:
            sub_category.at[index, i] = 1
        else:
            sub_category.at[index, i] = 0     
            
for i in sub:
    sub_category= sub_category.drop(columns=[i+"_x"])
    sub_category= sub_category.drop(columns=[i+"_y"])
    
df_num_1 = sub_category._get_numeric_data()
df_num_1_1 = df_num_1
df_num_2 = rest_9._get_numeric_data()
df_num_2_1 = df_num_2.drop(columns=['all_text_clean', 'all_text_clean_spacy', 'Unnamed: 4'])


'''combine lists'''
df_num_1 = df_num_1_1.drop(columns=['PIMS_ID'])
df_num_2 = df_num_2_1.drop(columns=['PIMS_ID'])
sub_sub_1 = df_num_1.columns.tolist()
sub_sub_2 = df_num_2.columns.tolist()
sub_sub = sub_sub_1 + list(set(sub_sub_2) - set(sub_sub_1))

unmatched_items_1 = [d for d in sub_sub_1 if d not in sub_sub_2]
unmatched_items_2 = [d for d in sub_sub_2 if d not in sub_sub_1]

for col in unmatched_items_1:
    df_num_2_1[col] = 0
    
for col in unmatched_items_2:
    df_num_1_1[col] = 0
    
sub_category = df_num_1_1.merge(df_num_2_1, how='left', left_on=['PIMS_ID'], right_on=['PIMS_ID'])

for index, row in sub_category.iterrows():
    for i in sub_sub:          
        if row[i+"_x"] or row[i+"_y"]:
            sub_category.at[index, i] = 1
        else:
            sub_category.at[index, i] = 0     
            
for i in sub_sub:
    sub_category= sub_category.drop(columns=[i+"_x"])
    sub_category= sub_category.drop(columns=[i+"_y"])
    
sub_strategis_3 = sub_category

In [297]:
'''merge with main strategies'''
strategies = main_strategy_1.drop(columns=['all_text_clean', 'all_text_clean_spacy', 'main_strategy_1','category', 'cat'])
strategies_1 = pd.concat([strategies, sub_strategis_1], axis=1)

strategies = main_strategy_2.drop(columns=['all_text_clean', 'all_text_clean_spacy', 'main_strategy_2','category', 'cat'])
strategies_2 = pd.concat([strategies, sub_strategis_2], axis=1)

strategies = main_strategy_3.drop(columns=['all_text_clean', 'all_text_clean_spacy', 'main_strategy_3','category', 'cat'])
strategies_3 = pd.concat([strategies, sub_strategis_3], axis=1)

## Dictionaries

In [298]:
strategies = ['capacity_building', 'enabling', 'finance_economy',
       'food_and_agricultural_commodities', 'governance', 'law_regulation',
       'management_operation', 'mitigation_adaptation', 'monitor_inventory',
       'technology_innovation']

#chemicals_and_waste
capacity_building = []
enabling = []
finance_economy = []
food_and_agricultural_commodities = []
governance = []
law_regulation =[]
management_operation = []
mitigation_adaptation = []
monitor_inventory = []
technology_innovation = []

lists = [capacity_building,
enabling,
finance_economy,
food_and_agricultural_commodities,
governance,
law_regulation ,
management_operation ,
mitigation_adaptation ,
monitor_inventory ,
technology_innovation]

d = dict(zip(strategies, lists))
d_1 = dict(zip(strategies, lists))
d_2 = dict(zip(strategies, lists))
d_3 = dict(zip(strategies, lists))

In [299]:
for key, value in d.items():
    for index, row in strategies_1.iterrows():
        if row[key] == 1:
            for i in range(len(strategies_1.columns)): 
                if row[i] == 1:
                    if row.index[i] not in value:
                        value.append(row.index[i])
                        
    for index, row in strategies_2.iterrows():
        if row[key] == 1:
            for i in range(len(strategies_2.columns)): 
                if row[i] == 1:
                    if row.index[i] not in value:
                        value.append(row.index[i]) 
                        
    for index, row in strategies_3.iterrows():
        if row[key] == 1:
            for i in range(len(strategies_3.columns)): 
                if row[i] == 1:
                    if row.index[i] not in value:
                        value.append(row.index[i])
                        
for key, value in d.copy().items():
    for i in value:
        if i == key:
            value.remove(i)
            
with open(os.path.abspath(os.path.join('../..', 'data/processed/'))+"/strategy_dict.pkl", 'wb') as handle:
    pickle.dump(d, handle, protocol=pickle.HIGHEST_PROTOCOL)